## Data Lake vs Data Lakehouse: Implementação prática com Spark

![capa](images/capa.png)

### Evolução das Arquiteturas de Dados

![data_architectures](images/data_architectures.png)

> # **Data Lakehouse** é uma nova arquitetura de gerenciamento de dados aberta que implementa estruturas de dados e recursos de gerenciamento de dados semelhantes aos de um Data Warehouse, diretamente no tipo de armazenamento de baixo custo usado para Data Lakes. 

![acid](images/acid.png)

In [1]:
!pip install pyspark==3.0.0

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import pyspark

spark = pyspark.sql.SparkSession.builder.appName("Lakehouse") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.databricks.delta.schema.autoMerge.enabled","true") \
    .config("spark.databricks.delta.autoOptimize.optimizeWrite","true") \
    .config("spark.databricks.delta.optimizeWrite.enabled","true") \
    .config("spark.databricks.delta.vacuum.parallelDelete.enabled","true") \
    .getOrCreate()

from delta.tables import *
from pyspark.sql.functions import *

In [3]:
spark

In [4]:
path = 'tmp/sample.parquet'

In [5]:
!rm -rf tmp/

**Criando uma tabela Delta**

Para criar uma tabela Delta, escreva um DataFrame no formato delta. Você pode usar o código Spark SQL existente e alterar o formato de parquet, csv, json e assim por diante para delta.

In [6]:
df_star_wars = spark.createDataFrame(
    [
        (1, 'Luke Skywalker', 1.72,'azul','19BBY','masculino','Tatooine','Humano'),
        (2, 'C-3PO',1.67,'amarelo','112BBY','NA','Tatooine','Droid'),
        (3, 'R2-D2', 0.67, 'vermelho','33BBY','NA','Naboo','Droid'),
        (4, 'Anakin Skywalker', 1.88, 'azul','41.9BBY','masculino','Tatooine','Humano'),
        (5, 'Leia Organa', 1.50,'castanho','19BBY','feminino','Alderaan','Humano'),
        (6, 'Han Solo', 1.80, 'castanho', '29BBY', 'masculino', 'Corellia', 'Humano'),
        (7, 'Yoda', 0.66, 'castanho', '896BBY', 'masculino', None, 'Yoda Especie')
     
    ],
        ['id', 'nome', 'altura', 'cor_dos_olhos','data_nascimento','sexo','planeta','especie']
)

df_star_wars.show()

+---+----------------+------+-------------+---------------+---------+--------+------------+
| id|            nome|altura|cor_dos_olhos|data_nascimento|     sexo| planeta|     especie|
+---+----------------+------+-------------+---------------+---------+--------+------------+
|  1|  Luke Skywalker|  1.72|         azul|          19BBY|masculino|Tatooine|      Humano|
|  2|           C-3PO|  1.67|      amarelo|         112BBY|       NA|Tatooine|       Droid|
|  3|           R2-D2|  0.67|     vermelho|          33BBY|       NA|   Naboo|       Droid|
|  4|Anakin Skywalker|  1.88|         azul|        41.9BBY|masculino|Tatooine|      Humano|
|  5|     Leia Organa|   1.5|     castanho|          19BBY| feminino|Alderaan|      Humano|
|  6|        Han Solo|   1.8|     castanho|          29BBY|masculino|Corellia|      Humano|
|  7|            Yoda|  0.66|     castanho|         896BBY|masculino|    null|Yoda Especie|
+---+----------------+------+-------------+---------------+---------+--------+--

In [7]:
df_star_wars.write.format("delta").save(path)

**Leitura dos Dados**

Para realizar a leitura de uma tabela Delta especificando o caminho para os arquivos: 

"/tmp/delta-table":

In [8]:
df_delta = spark.read.format("delta").load(path)
df_delta.show(truncate=False)

+---+----------------+------+-------------+---------------+---------+--------+------------+
|id |nome            |altura|cor_dos_olhos|data_nascimento|sexo     |planeta |especie     |
+---+----------------+------+-------------+---------------+---------+--------+------------+
|4  |Anakin Skywalker|1.88  |azul         |41.9BBY        |masculino|Tatooine|Humano      |
|5  |Leia Organa     |1.5   |castanho     |19BBY          |feminino |Alderaan|Humano      |
|1  |Luke Skywalker  |1.72  |azul         |19BBY          |masculino|Tatooine|Humano      |
|6  |Han Solo        |1.8   |castanho     |29BBY          |masculino|Corellia|Humano      |
|7  |Yoda            |0.66  |castanho     |896BBY         |masculino|null    |Yoda Especie|
|2  |C-3PO           |1.67  |amarelo      |112BBY         |NA       |Tatooine|Droid       |
|3  |R2-D2           |0.67  |vermelho     |33BBY          |NA       |Naboo   |Droid       |
+---+----------------+------+-------------+---------------+---------+--------+--

**Atualização condicional sem sobrescrever**

Delta Lake fornece APIs programáticas para atualização condicional, exclusão e mesclagem (upsert) de dados em tabelas. Aqui estão alguns exemplos.

In [9]:
df_star_wars_new = spark.createDataFrame(
    [
        (1, 'Luke Skywalker', 1.72,'azul','19BBY','masculino','Tatooine','Humano'),
        (2, 'C-3PO',1.67,'amarelo','112BBY','NA','Tatooine','Droid'),
        (3, 'R2-D2', 0.67, 'vermelho','33BBY','NA','Naboo','Droid'),
        (4, 'Darth Vader', 2.02, 'azul','41.9BBY','amarelo','Tatooine','Humano'),
        (5, 'Leia Organa', 1.50,'castanho','19BBY','feminino','Alderaan','Humano'),
        (6, 'Han Solo', 1.80, 'castanho', '29BBY', 'masculino', 'Corellia', 'Humano'),
        (7, 'Yoda', 0.66, 'castanho', '896BBY', 'masculino', None, 'Yoda Especie'),
        (8, 'Chewbacca', 2.28, 'azul', '200BBY', 'masculino', 'Kashyyyk', 'Wookiee'),
        (9, 'Boba Fett', 1.83, 'castanho', '31.5BBY', 'masculino', 'Kamino', 'Humano'),
        (10, 'Palpatine', 1.70, 'amarelo', '82BBY', 'masculino', 'Naboo', 'Humano'),
        
     
    ],
        ['id', 'nome', 'altura', 'cor_dos_olhos','data_nascimento','sexo','planeta','especie']
)

df_star_wars_new.show()

+---+--------------+------+-------------+---------------+---------+--------+------------+
| id|          nome|altura|cor_dos_olhos|data_nascimento|     sexo| planeta|     especie|
+---+--------------+------+-------------+---------------+---------+--------+------------+
|  1|Luke Skywalker|  1.72|         azul|          19BBY|masculino|Tatooine|      Humano|
|  2|         C-3PO|  1.67|      amarelo|         112BBY|       NA|Tatooine|       Droid|
|  3|         R2-D2|  0.67|     vermelho|          33BBY|       NA|   Naboo|       Droid|
|  4|   Darth Vader|  2.02|         azul|        41.9BBY|  amarelo|Tatooine|      Humano|
|  5|   Leia Organa|   1.5|     castanho|          19BBY| feminino|Alderaan|      Humano|
|  6|      Han Solo|   1.8|     castanho|          29BBY|masculino|Corellia|      Humano|
|  7|          Yoda|  0.66|     castanho|         896BBY|masculino|    null|Yoda Especie|
|  8|     Chewbacca|  2.28|         azul|         200BBY|masculino|Kashyyyk|     Wookiee|
|  9|     

In [10]:
table = DeltaTable.forPath(spark, path)

In [11]:
table.toDF().show(truncate=False)

+---+----------------+------+-------------+---------------+---------+--------+------------+
|id |nome            |altura|cor_dos_olhos|data_nascimento|sexo     |planeta |especie     |
+---+----------------+------+-------------+---------------+---------+--------+------------+
|4  |Anakin Skywalker|1.88  |azul         |41.9BBY        |masculino|Tatooine|Humano      |
|5  |Leia Organa     |1.5   |castanho     |19BBY          |feminino |Alderaan|Humano      |
|1  |Luke Skywalker  |1.72  |azul         |19BBY          |masculino|Tatooine|Humano      |
|6  |Han Solo        |1.8   |castanho     |29BBY          |masculino|Corellia|Humano      |
|7  |Yoda            |0.66  |castanho     |896BBY         |masculino|null    |Yoda Especie|
|2  |C-3PO           |1.67  |amarelo      |112BBY         |NA       |Tatooine|Droid       |
|3  |R2-D2           |0.67  |vermelho     |33BBY          |NA       |Naboo   |Droid       |
+---+----------------+------+-------------+---------------+---------+--------+--

In [12]:
table.alias("persisteddata") .merge( \
   df_star_wars_new.alias("newdata"), \
    "persisteddata.id = newdata.id") \
.whenMatchedUpdateAll() \
.whenNotMatchedInsertAll() \
.execute()

In [13]:
table.toDF().show(truncate=False)

+---+--------------+------+-------------+---------------+---------+--------+------------+
|id |nome          |altura|cor_dos_olhos|data_nascimento|sexo     |planeta |especie     |
+---+--------------+------+-------------+---------------+---------+--------+------------+
|5  |Leia Organa   |1.5   |castanho     |19BBY          |feminino |Alderaan|Humano      |
|1  |Luke Skywalker|1.72  |azul         |19BBY          |masculino|Tatooine|Humano      |
|9  |Boba Fett     |1.83  |castanho     |31.5BBY        |masculino|Kamino  |Humano      |
|6  |Han Solo      |1.8   |castanho     |29BBY          |masculino|Corellia|Humano      |
|8  |Chewbacca     |2.28  |azul         |200BBY         |masculino|Kashyyyk|Wookiee     |
|4  |Darth Vader   |2.02  |azul         |41.9BBY        |amarelo  |Tatooine|Humano      |
|10 |Palpatine     |1.7   |amarelo      |82BBY          |masculino|Naboo   |Humano      |
|7  |Yoda          |0.66  |castanho     |896BBY         |masculino|null    |Yoda Especie|
|2  |C-3PO

In [14]:
table.toDF().printSchema()

root
 |-- id: long (nullable = true)
 |-- nome: string (nullable = true)
 |-- altura: double (nullable = true)
 |-- cor_dos_olhos: string (nullable = true)
 |-- data_nascimento: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- planeta: string (nullable = true)
 |-- especie: string (nullable = true)



#### UPDATE DATA

In [15]:
deltaTable = DeltaTable.forPath(spark, path)

deltaTable.update("id = 4", { "cor_dos_olhos": "'amarelo'", "sexo": "'masculino'"} )

In [16]:
table.toDF().show(truncate=False)

+---+--------------+------+-------------+---------------+---------+--------+------------+
|id |nome          |altura|cor_dos_olhos|data_nascimento|sexo     |planeta |especie     |
+---+--------------+------+-------------+---------------+---------+--------+------------+
|4  |Darth Vader   |2.02  |amarelo      |41.9BBY        |masculino|Tatooine|Humano      |
|5  |Leia Organa   |1.5   |castanho     |19BBY          |feminino |Alderaan|Humano      |
|1  |Luke Skywalker|1.72  |azul         |19BBY          |masculino|Tatooine|Humano      |
|9  |Boba Fett     |1.83  |castanho     |31.5BBY        |masculino|Kamino  |Humano      |
|6  |Han Solo      |1.8   |castanho     |29BBY          |masculino|Corellia|Humano      |
|8  |Chewbacca     |2.28  |azul         |200BBY         |masculino|Kashyyyk|Wookiee     |
|10 |Palpatine     |1.7   |amarelo      |82BBY          |masculino|Naboo   |Humano      |
|7  |Yoda          |0.66  |castanho     |896BBY         |masculino|null    |Yoda Especie|
|2  |C-3PO

## DELETE DATA

In [17]:
table = DeltaTable.forPath(spark, path)
table.delete("id = 9")  

In [18]:
table = DeltaTable.forPath(spark, path)
table.toDF().show(truncate=False)

+---+--------------+------+-------------+---------------+---------+--------+------------+
|id |nome          |altura|cor_dos_olhos|data_nascimento|sexo     |planeta |especie     |
+---+--------------+------+-------------+---------------+---------+--------+------------+
|4  |Darth Vader   |2.02  |amarelo      |41.9BBY        |masculino|Tatooine|Humano      |
|5  |Leia Organa   |1.5   |castanho     |19BBY          |feminino |Alderaan|Humano      |
|1  |Luke Skywalker|1.72  |azul         |19BBY          |masculino|Tatooine|Humano      |
|6  |Han Solo      |1.8   |castanho     |29BBY          |masculino|Corellia|Humano      |
|8  |Chewbacca     |2.28  |azul         |200BBY         |masculino|Kashyyyk|Wookiee     |
|10 |Palpatine     |1.7   |amarelo      |82BBY          |masculino|Naboo   |Humano      |
|7  |Yoda          |0.66  |castanho     |896BBY         |masculino|null    |Yoda Especie|
|2  |C-3PO         |1.67  |amarelo      |112BBY         |NA       |Tatooine|Droid       |
|3  |R2-D2

## Histórico da Tabela Delta

In [19]:
df_star_wars = spark.read.format("delta").option("versionAsOf", 0).load(path)
df_star_wars.show()

+---+----------------+------+-------------+---------------+---------+--------+------------+
| id|            nome|altura|cor_dos_olhos|data_nascimento|     sexo| planeta|     especie|
+---+----------------+------+-------------+---------------+---------+--------+------------+
|  4|Anakin Skywalker|  1.88|         azul|        41.9BBY|masculino|Tatooine|      Humano|
|  5|     Leia Organa|   1.5|     castanho|          19BBY| feminino|Alderaan|      Humano|
|  1|  Luke Skywalker|  1.72|         azul|          19BBY|masculino|Tatooine|      Humano|
|  6|        Han Solo|   1.8|     castanho|          29BBY|masculino|Corellia|      Humano|
|  7|            Yoda|  0.66|     castanho|         896BBY|masculino|    null|Yoda Especie|
|  2|           C-3PO|  1.67|      amarelo|         112BBY|       NA|Tatooine|       Droid|
|  3|           R2-D2|  0.67|     vermelho|          33BBY|       NA|   Naboo|       Droid|
+---+----------------+------+-------------+---------------+---------+--------+--

In [20]:
df_star_wars = spark.read.format("delta").option("versionAsOf", 1).load(path)
df_star_wars.show()

+---+--------------+------+-------------+---------------+---------+--------+------------+
| id|          nome|altura|cor_dos_olhos|data_nascimento|     sexo| planeta|     especie|
+---+--------------+------+-------------+---------------+---------+--------+------------+
|  5|   Leia Organa|   1.5|     castanho|          19BBY| feminino|Alderaan|      Humano|
|  1|Luke Skywalker|  1.72|         azul|          19BBY|masculino|Tatooine|      Humano|
|  9|     Boba Fett|  1.83|     castanho|        31.5BBY|masculino|  Kamino|      Humano|
|  6|      Han Solo|   1.8|     castanho|          29BBY|masculino|Corellia|      Humano|
|  8|     Chewbacca|  2.28|         azul|         200BBY|masculino|Kashyyyk|     Wookiee|
|  4|   Darth Vader|  2.02|         azul|        41.9BBY|  amarelo|Tatooine|      Humano|
| 10|     Palpatine|   1.7|      amarelo|          82BBY|masculino|   Naboo|      Humano|
|  7|          Yoda|  0.66|     castanho|         896BBY|masculino|    null|Yoda Especie|
|  2|     

In [21]:
deltaTable = DeltaTable.forPath(spark, path)

In [22]:
deltaTable.show()

AttributeError: 'DeltaTable' object has no attribute 'show'